## PSO - Particle Swarm Optimisation

**About PSO -** 

PSO is an biologically inspired meta-heuristic optimisation algorithm. It takes its inspiration from bird flocking or fish schooling. It works pretty good in practice. So let us code it up and optimise a function.

In [2]:
#dependencies
import random
import math
import copy # for array copying
import sys

### COST Function

So basically the function we are trying to optimise will become our cost function.

What cost functions we will see:

1. Sum of squares
2. Rastrigin's function

### Rastrigins function:


Rastrgins equation:
![Rastrigins equation](images/rastrigins.svg)


3-D Rendering
![Rastrigin 3-d](images/Rastrigin_function.png)


As you can see its a non-convex function with lot of local minimas (i.e multi-modal : lot of optimal solutions). It is a fairly diffucult problem for testing and we will test this out.

In [3]:
# lets code the rastrigins function

def error(position):
  err = 0.0
  for i in range(len(position)):
    xi = position[i]
    err += (xi * xi) - (10 * math.cos(2 * math.pi * xi))
  err = 10*len(position) + err
  return err

### Particle

A particle basically maintains the following params:

1. particle position
2. particle velocity
3. best position individual
4. best error individual
5. error individual

The action it can take when traversing over its search space looks like -
```
Update velocity - 
w1*towards_current_direction(intertia) + w2*towards_self_best + w3*towards_swarm_best

Update position - 
Add current_velocity to previous_postion to obtain new_velocity

```
Now suppose the particle finds some minima/maxima which is better than the global best it has to update the global value. So we have its fitness evaluation function - 

```
evaluate fitness -
plug in current_position into test function to get where exactly you are that will give you the minima/maxima value
check against the global minima/maxima whether yours is better
assign value to global accordingly

```

In [4]:
# let us construct the class Particle

class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i=costFunc(self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.uniform(-1,1)
            r2=random.uniform(-1,1)

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]

### __PSO__ (Particle Swarm Optimisation)

In particle swarm optimisation we 
1. Initialise a swarm of particles to go on random exploration
2. for each particle we find whether the have discovered any new minima/maxima
3. The overall groups orientation or their velocities is guided to the global minimas

In [10]:
# Now let us define a class PSO
class PSO():
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('\nFINAL:')
        print (pos_best_g)
        print (err_best_g)

In [11]:
%time
initial=[5,5]               # initial starting location [x1,x2...]
bounds=[(-10,10),(-10,10)]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
PSO(error,initial,bounds,num_particles=15,maxiter=30)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs

FINAL:
[4.974662731327702, 4.974690991575899]
49.747446030022026


Now further on we will try to parallelize PSO algorithm